In [58]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from datetime import datetime
import re
browser = webdriver.Chrome()
from selenium.common.exceptions import NoSuchElementException

conn=pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='tldkswndlssla',  # 데이터베이스 비밀번호
    db='YES24',       # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:


    link_list=[]     
        #got the link of searching "python", and iterated with for statement to get the each page.
        # the goal was get the elements of books to get the links for each book in the next step.
    for page in range(1,4):
        browser.get(f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page}&pageSize=24')
        inform = browser.find_elements(By.CLASS_NAME, 'gd_name')
    # used for statement to iterate each book element to get the each book link and add to list.
    # this will be used to get the informations such as title, publisher, etc in the next step. 

        for book_link in inform:
            link = book_link.get_attribute('href')
            link_list.append(link)
            # time.sleep(0.1)

                
    for link in link_list:
        browser.get(link)


        try:
            title = browser.find_element(By.CLASS_NAME, 'gd_name').text
            publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
            publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
            match = re.match(r'(\d+)년 (\d+)월 (\d+)일',publishing)
            if match:
                year, month, day = match.groups()
                data_obj = datetime(int(year), int(month), int(day))
                publishing = data_obj.strftime("%Y-%m-%d")
            else:
                publishing  = "2023-01-01"
            author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
            rating = browser.find_element(By.CLASS_NAME, 'gd_rating').find_element(By.CLASS_NAME, 'yes_b').text
            rating = float(rating.replace(",",""))  
            review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.CLASS_NAME, 'txC_blue').text
            review=int(review.replace(",",""))
            sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
            sales=int(sales.replace(",",""))
            price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
            price=float(price.replace(",",""))    
            ranking = browser.find_element(By.CLASS_NAME, 'gd_best').find_element(By.TAG_NAME, 'a').text
            match = re.search(r'\d+', ranking)
            if match:
                ranking = int(match.group())  # Convert the found digits to an integer
            else:
                ranking = None 
            ranking_weeks_element = browser.find_element(By.CLASS_NAME, 'gd_best_tp02').text.split(" | ")[1]
            match = re.search(r'(\d+) (\d+)', ranking_weeks_element)
            if match:
                ranking_weeks_element = int(match.group(2))  # Convert the found digits to an integer
            else:
                ranking_weeks_element = None 
            
        except Exception or NoSuchElementException as e:
            rating = 0.0
            review = 0
            sales = 0
            ranking = 0
            ranking_weeks_element = 

        
        print(ranking_weeks_element)

        sql = """INSERT INTO BOOKS (title, author, publisher, publishing, rating, 
                                review, sales, price, ranking, ranking_weeks)
                                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cur.execute(sql, (title, author, publisher, publishing,rating,review,ranking,ranking_weeks,price,sales))
        conn.commit()
        


None
1
1
None
5
1
None
1
None
5
None
17
3
None
4


KeyboardInterrupt: 